In [1]:
import requests # 서버 요청
from bs4 import BeautifulSoup as bs #html 파싱  
import time # 5초간 휴식 할때 사용  
import pandas as pd # 데이터프레임 
from datetime import datetime # 오늘 연월일 추출 

In [3]:
today = datetime.today() #현재 날짜 시간까지 뽑아줌
print(today.month)

9


In [4]:
yyyy = today.year
mm = today.month 

print(yyyy,mm)

2023 9


In [5]:
year = list(range(2023,2021,-1))
print(year)

month = list(range(1,13))
print(month)

page = list(range(1,10))
print(page)

[2023, 2022]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [11]:
bookList = [] 

def extract_bookinfo(yy,mon,book_list):
    for book in book_list:
        book_dict = {}
        title = book.select_one(".gd_name").text
        if book.select(".gd_nameE"):
            subtitle = book.select_one(".gd_nameE").text
        elif book.select(".gd_nameF"):
            subtitle = book.select_one(".gd_nameF").text
        else:
            subtitle = None
        link = "https://www.yes24.com/"+book.select_one(".gd_name")['href'].strip()
        author = [i.get_text().strip() for i in book.select_one(".authPub.info_auth > a")] if  book.select_one(".authPub.info_auth > a") is not None else "None" 
        publisher = book.select_one(".authPub.info_pub > a").text.strip() if  book.select_one(".authPub.info_pub > a") is not None else "None"
        publish_date = book.select_one(".authPub.info_date").text.strip() if book.select_one(".authPub.info_date") is not None else "None"
        price = book.select_one(".txt_num .yes_b").text.strip() + "원" if book.select_one(".txt_num .yes_b" ) is not None else 0
        review = book.select_one(".rating_rvCount .txC_blue").text.strip() if book.select_one(".rating_rvCount .txC_blue") is not None else 0
        review_link = book.select_one(".rating_rvCount > a")['href'].strip() if book.select_one(".rating_rvCount > a") is not None else "None"
        star_pont = book.select_one(".rating_grade .yes_b").text.strip() if book.select_one(".rating_grade .yes_b") is not None else 0
        tag = [i.get_text().strip() for i in book.select(".info_row.info_tag > .tag > a")] if book.select(".info_row.info_tag > .tag > a") is not None else "None" 

        book_dict["년"] = yy
        book_dict["월"] = mon
        book_dict["제목"] = title 
        book_dict["부제목"] =subtitle
        book_dict["링크"] =link 
        book_dict["작가"] = author
        book_dict["출판사"] = publisher
        book_dict["출판일"] = publish_date
        book_dict["가격"] = price
        book_dict["리뷰"] = review
        book_dict["리뷰링크"] = review_link
        book_dict["별점"] = star_pont
        book_dict["태그"] = tag

        bookList.append(book_dict)
    return bookList


def detail_page_info(url):
    detail_result = []
    for index, url2 in enumerate(url[:1]):
        detail_dict = {}
        print(f"{index}/{len(url)} 데이터 추출 중")
        r2 = requests.get(url2)
        soup2 = bs(r2.text, 'lxml')
        book_id = url2.split("/")[-1]
        if soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td") != None:
            if "쪽수" in soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > th").text:
                if len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")) == 3:
                    page, weight, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")
                elif len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")) == 2 and \
                       soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")[1][-2:] == "mm":
                    page, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")
                    weight = 0
            elif "쪽수" in soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > th").text:
                if len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")) == 3:
                    page, weight, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")
                elif len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")) == 2 and \
                   soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")[1][-2:] == "mm":
                    page, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")
                    weight = 0
            elif "쪽수" in soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > th").text:
                if len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")) == 3:
                    page, weight, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")
                elif len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")) == 2 and \
                       soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")[1][-2:] == "mm":
                        page, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")
                        weight = 0
        else:
            page = 0
            weight = 0
            size = ""
        category = list({i. text for i in soup2.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") if soup2.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") != None})
        book_intro = soup2.select_one(".infoWrap_txtInner").get_text() if soup2.select_one(".infoWrap_txtInner") != None else ""
        pub_book_intro = soup2.select_one(".infoWrap_txt").get_text() if soup2.select_one(".infoWrap_txt") != None else ""
        
        
        time.sleep(5) 
        detail_dict["ID"] = book_id
        detail_dict["페이지"] = page
        detail_dict["무게"] = weight
        detail_dict["사이즈"] = size
        detail_dict["카테고리"] = category
        detail_dict["책소개"] = book_intro
        detail_dict["출판사리뷰"] = pub_book_intro
        detail_result.append(detail_dict)  # 결과를 detail_result 리스트에 추가
        
    return detail_result

for yy in year[:1]:
    for mon in month if yy != yyyy else range(1, mm)[:1]:
        for p in page[:2]:
            print(f"{yy}년{mon}월{p}페이지 수집중", end="\r")
            url = "https://www.yes24.com/Product/Category/MonthWeekBestSeller"
            payload = dict(categoryNumber='001', pageNumber=p, pageSize=120, type='month', saleYear=yy, saleMonth=mon)
            
            try:
                r = requests.get(url, params=payload) # yes24 서버에 url로 자료 요청
            except Exception as e:
                print(e)
            
            soup = bs(r.text, 'lxml')   # yes24 서버에서 받은 응답 값 텍스트를 beautifulsoup을 통해 html로 파싱
            book_list = soup.select("#yesBestList > li")
            result = extract_bookinfo(yy, mon, book_list)  # extract_bookinfo 함수로 추출한 데이터를 result에 추가
            time.sleep(5)

df = pd.DataFrame(result)
df.to_csv("베스트셀러2023.csv",encoding="cp949",index=False)
df['book_id'] =  df['링크'].apply(lambda x: x.split("/")[-1]) 
data = pd.read_csv("C:/Users/UICHEOL_HWANG/Edu_file/my_python_lab/베스트셀러2023.csv",encoding="cp949")
urls = data['링크']

detail_result = detail_page_info(urls)  # 전체 URL 목록을 detail_page_info 함수에 전달
detail_df = pd.DataFrame(detail_result)
data_with_detail = pd.concat([data, detail_df], axis=1)

0/240 데이터 추출 중중


In [12]:
data_with_detail 

,년,월,제목,부제목,링크,작가,출판사,출판일,가격,리뷰,리뷰링크,별점,태그,ID,페이지,무게,사이즈,카테고리,책소개,출판사리뷰
0,2023,1,만일 내가 인생을 다시 산다면,벌써 마흔이 된 당신에게 해 주고 싶은 말들 42,https://www.yes24.com//Product/Goods/115142458,['김혜남'],메이븐,2022년 11월,"15,480원",133,https://www.yes24.com/Product/Goods/115142458?...,9.6,"['#2023대구올해의책', '#크레마클럽에있어요', '#인생선배', '#인생2막대...",115142458,280쪽,408g,145*210*18mm,"[주제로 읽는 인문학 , 인문, 국내도서, 노년 / 죽음]",\n\r\n“하나의 문이 닫히면 또 하나의 문이 열린다. \r\n그러니 더 이상 고...,\n\n\r\n“하나의 문이 닫히면 또 하나의 문이 열린다. \r\n그러니 더 이상...
1,2023,1,스카이 버스,명문 대학으로 직행하는 초등 공부 전략서,https://www.yes24.com//Product/Goods/116579452,['분당강쌤'],다산에듀,2023년 01월,"16,200원",244,https://www.yes24.com/Product/Goods/116579452?...,9.8,['#초등공부'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,1,생에 감사해,NaN,https://www.yes24.com//Product/Goods/116046489,['김혜자'],수오서재,2022년 12월,"15,300원",57,https://www.yes24.com/Product/Goods/116046489?...,9.6,"['#문학뉴스레터에소개된책', '#감사하는삶', '#삶을노래하다', '#배우의삶',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,1,불편한 편의점,NaN,https://www.yes24.com//Product/Goods/99308021,['김호연'],나무옆의자,2021년 04월,"12,600원",518,https://www.yes24.com/Product/Goods/99308021?R...,9.2,"['#2022포천시올해의책', '#2022광양시올해의책', '#크레마클럽에있어요',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023,1,불편한 편의점 2,NaN,https://www.yes24.com//Product/Goods/111088149,['김호연'],나무옆의자,2022년 08월,"12,600원",200,https://www.yes24.com/Product/Goods/111088149?...,9.2,"['#크레마클럽에있어요', '#가을에어울리는책', '#이웃의풍경', '#현실을그렸어...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2023,1,조셉 머피 부의 초월자,무한의 부를 창조하는 잠재의식의 힘,https://www.yes24.com//Product/Goods/114857120,['조셉 머피'],다산북스,2022년 10월,"19,800원",16,https://www.yes24.com/Product/Goods/114857120?...,9.7,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,2023,1,백년허리1 진단편,내 허리 통증 해석하기,https://www.yes24.com//Product/Goods/99360153,['정선근'],언탱글링,2021년 04월,"16,650원",24,https://www.yes24.com/Product/Goods/99360153?R...,9.5,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,2023,1,열린 어둠,NaN,https://www.yes24.com//Product/Goods/116421662,['렌죠 미키히코'],모모,2022년 12월,"15,120원",85,https://www.yes24.com/Product/Goods/116421662?...,9.6,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,2023,1,지적 대화를 위한 넓고 얕은 지식 제로,지혜를 찾아 138억 년을 달리는 시간 여행서,https://www.yes24.com//Product/Goods/84659792,['채사장'],웨일북,2019년 12월,"17,820원",166,https://www.yes24.com/Product/Goods/84659792?R...,9.6,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


https://www.yes24.com//Product/Goods/115142458
https://www.yes24.com//Product/Goods/116579452
https://www.yes24.com//Product/Goods/116046489
https://www.yes24.com//Product/Goods/99308021
https://www.yes24.com//Product/Goods/111088149
https://www.yes24.com//Product/Goods/112253263
https://www.yes24.com//Product/Goods/9349031
https://www.yes24.com//Product/Goods/116206629
https://www.yes24.com//Product/Goods/113416767
https://www.yes24.com//Product/Goods/116789002
https://www.yes24.com//Product/Goods/116206662
https://www.yes24.com//Product/Goods/116614181
https://www.yes24.com//Product/Goods/102360203
https://www.yes24.com//Product/Goods/111085946
https://www.yes24.com//Product/Goods/117014613
https://www.yes24.com//Product/Goods/108850617
https://www.yes24.com//Product/Goods/62921516
https://www.yes24.com//Product/Goods/105384881
https://www.yes24.com//Product/Goods/62921746
https://www.yes24.com//Product/Goods/62921720
https://www.yes24.com//Product/Goods/109705390
https://www.yes24.c

In [36]:
df['링크'].apply(lambda x: x.split("/")[-1]) 

0      115142458
1      116579452
2      116046489
3       99308021
4      111088149
         ...    
235    114857120
236     99360153
237    116421662
238     84659792
239     67033342
Name: 링크, Length: 240, dtype: object

In [26]:
df

,년,월,제목,부제목,링크,작가,출판사,출판일,가격,리뷰,리뷰링크,별점,태그
0,2023,1,만일 내가 인생을 다시 산다면,벌써 마흔이 된 당신에게 해 주고 싶은 말들 42,https://www.yes24.com//Product/Goods/115142458,[김혜남],메이븐,2022년 11월,"15,480원",133,https://www.yes24.com/Product/Goods/115142458?...,9.6,"[#2023대구올해의책, #크레마클럽에있어요, #인생선배, #인생2막대비, #내인생..."
1,2023,1,스카이 버스,명문 대학으로 직행하는 초등 공부 전략서,https://www.yes24.com//Product/Goods/116579452,[분당강쌤],다산에듀,2023년 01월,"16,200원",244,https://www.yes24.com/Product/Goods/116579452?...,9.8,[#초등공부]
2,2023,1,생에 감사해,None,https://www.yes24.com//Product/Goods/116046489,[김혜자],수오서재,2022년 12월,"15,300원",57,https://www.yes24.com/Product/Goods/116046489?...,9.6,"[#문학뉴스레터에소개된책, #감사하는삶, #삶을노래하다, #배우의삶, #연예인에세이]"
3,2023,1,불편한 편의점,None,https://www.yes24.com//Product/Goods/99308021,[김호연],나무옆의자,2021년 04월,"12,600원",518,https://www.yes24.com/Product/Goods/99308021?R...,9.2,"[#2022포천시올해의책, #2022광양시올해의책, #크레마클럽에있어요, #흩날리는..."
4,2023,1,불편한 편의점 2,None,https://www.yes24.com//Product/Goods/111088149,[김호연],나무옆의자,2022년 08월,"12,600원",200,https://www.yes24.com/Product/Goods/111088149?...,9.2,"[#크레마클럽에있어요, #가을에어울리는책, #이웃의풍경, #현실을그렸어요, #편의점에서]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2023,1,조셉 머피 부의 초월자,무한의 부를 창조하는 잠재의식의 힘,https://www.yes24.com//Product/Goods/114857120,[조셉 머피],다산북스,2022년 10월,"19,800원",16,https://www.yes24.com/Product/Goods/114857120?...,9.7,[]
236,2023,1,백년허리1 진단편,내 허리 통증 해석하기,https://www.yes24.com//Product/Goods/99360153,[정선근],언탱글링,2021년 04월,"16,650원",24,https://www.yes24.com/Product/Goods/99360153?R...,9.5,[]
237,2023,1,열린 어둠,None,https://www.yes24.com//Product/Goods/116421662,[렌죠 미키히코],모모,2022년 12월,"15,120원",85,https://www.yes24.com/Product/Goods/116421662?...,9.6,[]
238,2023,1,지적 대화를 위한 넓고 얕은 지식 제로,지혜를 찾아 138억 년을 달리는 시간 여행서,https://www.yes24.com//Product/Goods/84659792,[채사장],웨일북,2019년 12월,"17,820원",166,https://www.yes24.com/Product/Goods/84659792?R...,9.6,[]


In [7]:
for yy in year:
    for mon in month:
        for p in page:
            print(yy,mon,p)

2023 1 1
2023 1 2
2023 1 3
2023 1 4
2023 1 5
2023 1 6
2023 1 7
2023 1 8
2023 1 9
2023 2 1
2023 2 2
2023 2 3
2023 2 4
2023 2 5
2023 2 6
2023 2 7
2023 2 8
2023 2 9
2023 3 1
2023 3 2
2023 3 3
2023 3 4
2023 3 5
2023 3 6
2023 3 7
2023 3 8
2023 3 9
2023 4 1
2023 4 2
2023 4 3
2023 4 4
2023 4 5
2023 4 6
2023 4 7
2023 4 8
2023 4 9
2023 5 1
2023 5 2
2023 5 3
2023 5 4
2023 5 5
2023 5 6
2023 5 7
2023 5 8
2023 5 9
2023 6 1
2023 6 2
2023 6 3
2023 6 4
2023 6 5
2023 6 6
2023 6 7
2023 6 8
2023 6 9
2023 7 1
2023 7 2
2023 7 3
2023 7 4
2023 7 5
2023 7 6
2023 7 7
2023 7 8
2023 7 9
2023 8 1
2023 8 2
2023 8 3
2023 8 4
2023 8 5
2023 8 6
2023 8 7
2023 8 8
2023 8 9
2023 9 1
2023 9 2
2023 9 3
2023 9 4
2023 9 5
2023 9 6
2023 9 7
2023 9 8
2023 9 9
2023 10 1
2023 10 2
2023 10 3
2023 10 4
2023 10 5
2023 10 6
2023 10 7
2023 10 8
2023 10 9
2023 11 1
2023 11 2
2023 11 3
2023 11 4
2023 11 5
2023 11 6
2023 11 7
2023 11 8
2023 11 9
2023 12 1
2023 12 2
2023 12 3
2023 12 4
2023 12 5
2023 12 6
2023 12 7
2023 12 8
2023 12 9
2

In [ ]:
def to_db(df):
    for col in df.columns:
        df[col] = df[col].apply(str)    
    engine = create_engine('sqlite:///yes24_final.db', echo= False)
    conn = engine.raw_connection()
    df.to_sql('yes24_final_fn', con=conn, if_exists='append')
    print("데이터베이스 저장 완료")

# 함수 적용 전 코드 

In [ ]:
bookList = []

year = list(range(2023, 2021, -1))
month = list(range(1, 13))
page = list(range(1, 10))

for yy in year[:1]:
    for mon in (month if yy != yyyy else range(1, mm)):
        for p in page[:2]:
            print(f"{yy}년 {mon}월 {p}페이지 수집중", end="\r")
            url = "https://www.yes24.com/Product/Category/MonthWeekBestSeller"
            payload = dict(categoryNumber="001", pageNumber=p, pageSize="120", type="month", saleYear=yy, saleMonth=mon)
            
            try :
                r = requests.get(url,params=payload) # yes24 서버에 url로 자료요청 
            except Exception as e :
                print(e)
            
            soup = BeautifulSoup(r.text, 'lxml')
            book_list = soup.select("#yesBestList > li")  # 각 페이지마다 새로운 데이터를 추출
            
            for book in book_list:
                book_dict = {}
                title = book.select_one(".gd_name").text
                if book.select(".gd_nameE"):
                    subtitle = book.select_one(".gd_nameE").text
                elif book.select(".gd_nameF"):
                    subtitle = book.select_one(".gd_nameF").text
                else:
                    subtitle = None
                link = "https://www.yes24.com/" + book.select_one(".gd_name")['href'].strip()
                author = [i.get_text().strip() for i in book.select_one(".authPub.info_auth > a")] if book.select_one(".authPub.info_auth > a") is not None else "None"
                publisher = book.select_one(".authPub.info_pub > a").text.strip()
                publish_date = book.select_one(".authPub.info_date").text.strip()
                price = book.select_one(".txt_num .yes_b").text.strip() + "원" if book.select_one(".txt_num .yes_b") is not None else "0"
                review = book.select_one(".rating_rvCount .txC_blue").text.strip() if book.select_one(".rating_rvCount .txC_blue") is not None else "0"
                review_link = book.select_one(".rating_rvCount > a")['href'].strip() if book.select_one(".rating_rvCount > a") is not None else "None"
                star_point = book.select_one(".rating_grade .yes_b").text.strip()
                tag = [i.get_text().strip() for i in book.select(".info_row.info_tag > .tag > a")] if book.select(".info_row.info_tag > .tag > a") is not None else "None"

                book_dict["년"] = yy
                book_dict["월"] = mon
                book_dict["제목"] = title
                book_dict["부제목"] = subtitle
                book_dict["링크"] = link
                book_dict["작가"] = author
                book_dict["출판사"] = publisher
                book_dict["출판일"] = publish_date
                book_dict["가격"] = price
                book_dict["리뷰"] = review
                book_dict["리뷰링크"] = review_link
                book_dict["별점"] = star_point
                book_dict["태그"] = tag

                bookList.append(book_dict)
            time.sleep(5)


In [ ]:
import pandas as pd 


df = pd.read_csv("C:/Users/hi/Desktop/bigdata/예스24_월간베스트_2022,2023.csv",encoding="utf-8")

linkList = df['링크']

In [ ]:
df.to_csv("예스24_월간베스트_2022,2023.csv",encoding="utf-8")

## DB 저장 
* pandas 데이터프레임을 DB에 바로 저장하기

In [ ]:
df1 = pd.read_csv("C:/Users/hi/Desktop/bigdata/예스24_월간베스트_2022,2023.csv",encoding="utf-8")


In [ ]:
df1.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
df1

# pandas를 이용해서 datatype를 변경 

In [ ]:
detail_df["무게"] = detail_df["무게"].apply(str)
detail_df["페이지"] = detail_df["페이지"].apply(str)
detail_df["카테고리"] = detail_df["카테고리"].apply(str)

In [ ]:
import sqlite3 

con =sqlite3.connect("sqlite3_yes24_best2023_01_detail.db")

cursor = con.cursor()

In [ ]:
df1.columns

In [ ]:
detail_df.columns

In [ ]:
columns = list(df1.columns)
columns_str = ', '.join(f"'{col}' TEXT" for col in columns)
query = f'CREATE TABLE IF NOT EXISTS yes24_best_202301_detail ({columns_str})'
cursor.execute(query)

In [ ]:
detail_df.to_sql('yes24_best_202301_detail',con,if_exists='replace',index=False)

In [ ]:
cursor.execute(
'''
    SELECT * FROM yes24_best_202301_detail limit 100;
''')

for row in cursor.fetchall():
    print(row)

## sqlalchemy를 이용하는 방법 

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///test.db',echo=False)
conn = engine.raw_connection()
df1.to_sql('yes24_sqlalchemy',con=conn,if_exists='append')


In [ ]:
test = pd.read_sql('SELECT * FROM yes24_sqlalchemy limit 10',conn)

test

In [ ]:
conn.close()